In [ ]:
import openai
import pandas as pd
import numpy as np
import ast
from getpass import getpass

#api key^
api_key = 
openai.api_key = api_key

In [5]:
#embedding function without the file path stuff
# Load your CSV file
data = pd.read_csv('./fomcminutes20220126_cleaned.csv')

# Function to fetch embeddings from OpenAI
def get_embeddings(text):
    try:
        response = openai.Embedding.create(
            input=text,
            engine="text-embedding-3-small"  # Check the latest documentation for available engines
        )
        return response['data'][0]['embedding']  # Extracting the embedding vector
    except Exception as e:
        return str(e)  # Return error as string for debugging

# Apply the function to each text entry in the CSV and create a new column for embeddings
data['embeddings'] = data['text'].apply(get_embeddings)

# Save the DataFrame with the embeddings to a new CSV file
data.to_csv('FedSpeechMay1_with_embeddings.csv', index=False)

# Printing a message to confirm completion
print("CSV file with embeddings saved successfully.")

CSV file with embeddings saved successfully.


In [6]:
# Load the CSV file
data = pd.read_csv('./FedSpeechMay1_with_embeddings.csv')

# Convert string representations of lists to actual lists of floats
data['embeddings'] = data['embeddings'].apply(ast.literal_eval)

# Optional: Convert lists to NumPy arrays for easier numerical manipulation
data['embeddings'] = data['embeddings'].apply(np.array)

# Print the first few rows to check the embeddings

data.to_csv('Updated2Speech.csv', index=False)
print('embedding column now in array format')

embedding column now in array format


In [7]:
type(data.iloc[0].embeddings)

numpy.ndarray

In [8]:
search_term = input('Enter a Search Term:')
search_term_vector = get_embeddings(search_term)
search_term_vector

Enter a Search Term: economy


[0.010721882805228233,
 -0.014457287266850471,
 0.0180292297154665,
 -0.014263554476201534,
 0.036058459430933,
 0.010201225988566875,
 -0.006738258060067892,
 0.10752153396606445,
 -0.030488651245832443,
 -0.029883235692977905,
 0.03651857376098633,
 -0.035549912601709366,
 -0.019094757735729218,
 -0.017484357580542564,
 0.015631789341568947,
 0.031602613627910614,
 -0.03540461137890816,
 -0.018477236852049828,
 -0.009075156413018703,
 3.913907130481675e-05,
 0.015316975302994251,
 0.002810634672641754,
 -0.012447312474250793,
 -0.005270129069685936,
 0.045575566589832306,
 -0.002028136979788542,
 -0.020281370729207993,
 0.0109943188726902,
 0.006108627654612064,
 -0.03361258655786514,
 0.015547032468020916,
 -0.04438895732164383,
 0.033806320279836655,
 0.0038110814057290554,
 0.025500038638710976,
 -0.006490038242191076,
 -0.007119669113308191,
 0.004253033548593521,
 0.058604076504707336,
 -0.04877215251326561,
 -0.010061981156468391,
 -0.02712254785001278,
 -0.014578369446098804,


In [9]:
def cosine_similarity(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    if np.linalg.norm(vec1) == 0 or np.linalg.norm(vec2) == 0:
        return 0  # Avoid division by zero
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

data['similarities'] = data['embeddings'].apply(lambda x: cosine_similarity(x, search_term_vector))

# Displaying the DataFrame
print(data.head(5))

                                                text  \
0  Minutes of the Federal Open Market Committee J...   
1  and continued on Wednesday, January 26, 2022, ...   
2                               Powell, Chair John C   
3                    Williams, Vice Chair Michelle W   
4        Bowman Lael Brainard James Bullard Esther L   

                                          embeddings  similarities  
0  [-0.012299942784011364, 0.008014765568077564, ...      0.172105  
1  [-0.011248289607465267, 0.03397614136338234, 0...      0.074953  
2  [0.008248341269791126, 0.013346750289201736, 0...      0.139306  
3  [0.024420615285634995, 0.021282630041241646, 0...      0.054053  
4  [0.02811501733958721, -0.024600639939308167, 0...      0.112836  


In [10]:
pd.set_option('display.max_colwidth', None)
data.sort_values("similarities", ascending=False).text[:20]

411                                                                                                                                                                                                                                                                                                                                                             economy
421                                                                                                                                                                                                                                                                                              The path of the economy continues to depend on the course of the virus
401                                                                                                                                                                                                                                                              Mem-bers also acknowled

In [11]:
data.sort_values("similarities", ascending=False).similarities[:20]

411    1.000000
421    0.454581
401    0.440238
324    0.438721
318    0.385117
307    0.378448
107    0.365810
395    0.360207
135    0.360075
354    0.359673
136    0.352822
49     0.350929
308    0.347698
383    0.342013
420    0.335331
400    0.335331
328    0.333111
386    0.322927
240    0.319766
22     0.317962
Name: similarities, dtype: float64